# Esercizio 4 - Soluzione

### 1) Parametri in input

In [1]:
embl_file_name = './M10051.embl'
genetic_code_name = './genetic-code.txt'

### 2) Importazione del modulo `re`

In [2]:
import re

### 3) Definizione di una funzione per formattare in `FASTA` una sequenza

La funzione prende come argomenti un *header* `FASTA` (che inizia con un simbolo `>`), una sequenza nucleotidica (o di proteina) e la dimensione (passata come *keyword* `record` con valore di *default* pari a 80) dei *record* in cui separare la sequenza. La funzione restituisce una stringa contenente la sequenza in formato `FASTA`.

*Suggerimento*: usare la funzione `compile()` per creare un'espressione regolare.

In [3]:
def format_fasta(header, sequence, record = 80):    
    p = re.compile('\w{,' + str(record) + '}')
    return header + '\n' + '\n'.join(re.findall(p, sequence))

### 4) Lettura del file del codice genetico in una lista di righe

In [5]:
genetic_code_rows

['A,gct,gcc,gca,gcg\n',
 'C,tgt,tgc\n',
 'D,gat,gac\n',
 'E,gaa,gag\n',
 'F,ttt,ttc\n',
 'G,ggt,ggc,gga,ggg\n',
 'H,cat,cac\n',
 'I,att,atc,ata\n',
 'K,aaa,aag\n',
 'L,tta,ttg,ctt,ctc,cta,ctg\n',
 'M,atg\n',
 'N,aat,aac\n',
 'P,cct,ccc,cca,ccg\n',
 'Q,caa,cag\n',
 'R,cgt,cgc,cga,cgg,aga,agg\n',
 'S,tct,tcc,tca,tcg,agt,agc\n',
 'T,act,acc,aca,acg\n',
 'V,gtt,gtc,gta,gtg\n',
 'W,tgg\n',
 'Y,tat,tac\n',
 's,tga,taa,tag']

### 5) Costruzione del dizionario del codice genetico

Costruire il dizionario che contiene il codice genetico:

- *chiave*: codone
- *valore*: simbolo dell'amminoacido codificato

a) Costruire la lista delle tuple *(chiave, valore)*.

*Suggerimento*: usare la funzione `product()` del modulo `itertools` che effettua il prodotto cartesiano tra le liste passate come argomento.

In [9]:
split_genetic_code

[['A', 'gct', 'gcc', 'gca', 'gcg'],
 ['C', 'tgt', 'tgc'],
 ['D', 'gat', 'gac'],
 ['E', 'gaa', 'gag'],
 ['F', 'ttt', 'ttc'],
 ['G', 'ggt', 'ggc', 'gga', 'ggg'],
 ['H', 'cat', 'cac'],
 ['I', 'att', 'atc', 'ata'],
 ['K', 'aaa', 'aag'],
 ['L', 'tta', 'ttg', 'ctt', 'ctc', 'cta', 'ctg'],
 ['M', 'atg'],
 ['N', 'aat', 'aac'],
 ['P', 'cct', 'ccc', 'cca', 'ccg'],
 ['Q', 'caa', 'cag'],
 ['R', 'cgt', 'cgc', 'cga', 'cgg', 'aga', 'agg'],
 ['S', 'tct', 'tcc', 'tca', 'tcg', 'agt', 'agc'],
 ['T', 'act', 'acc', 'aca', 'acg'],
 ['V', 'gtt', 'gtc', 'gta', 'gtg'],
 ['W', 'tgg'],
 ['Y', 'tat', 'tac'],
 ['s', 'tga', 'taa', 'tag']]

In [11]:
key_value_list

[('gct', 'A'),
 ('gcc', 'A'),
 ('gca', 'A'),
 ('gcg', 'A'),
 ('tgt', 'C'),
 ('tgc', 'C'),
 ('gat', 'D'),
 ('gac', 'D'),
 ('gaa', 'E'),
 ('gag', 'E'),
 ('ttt', 'F'),
 ('ttc', 'F'),
 ('ggt', 'G'),
 ('ggc', 'G'),
 ('gga', 'G'),
 ('ggg', 'G'),
 ('cat', 'H'),
 ('cac', 'H'),
 ('att', 'I'),
 ('atc', 'I'),
 ('ata', 'I'),
 ('aaa', 'K'),
 ('aag', 'K'),
 ('tta', 'L'),
 ('ttg', 'L'),
 ('ctt', 'L'),
 ('ctc', 'L'),
 ('cta', 'L'),
 ('ctg', 'L'),
 ('atg', 'M'),
 ('aat', 'N'),
 ('aac', 'N'),
 ('cct', 'P'),
 ('ccc', 'P'),
 ('cca', 'P'),
 ('ccg', 'P'),
 ('caa', 'Q'),
 ('cag', 'Q'),
 ('cgt', 'R'),
 ('cgc', 'R'),
 ('cga', 'R'),
 ('cgg', 'R'),
 ('aga', 'R'),
 ('agg', 'R'),
 ('tct', 'S'),
 ('tcc', 'S'),
 ('tca', 'S'),
 ('tcg', 'S'),
 ('agt', 'S'),
 ('agc', 'S'),
 ('act', 'T'),
 ('acc', 'T'),
 ('aca', 'T'),
 ('acg', 'T'),
 ('gtt', 'V'),
 ('gtc', 'V'),
 ('gta', 'V'),
 ('gtg', 'V'),
 ('tgg', 'W'),
 ('tat', 'Y'),
 ('tac', 'Y'),
 ('tga', 's'),
 ('taa', 's'),
 ('tag', 's')]

b) Costruire il dizionario.

In [13]:
genetic_code_dict

{'gct': 'A',
 'gcc': 'A',
 'gca': 'A',
 'gcg': 'A',
 'tgt': 'C',
 'tgc': 'C',
 'gat': 'D',
 'gac': 'D',
 'gaa': 'E',
 'gag': 'E',
 'ttt': 'F',
 'ttc': 'F',
 'ggt': 'G',
 'ggc': 'G',
 'gga': 'G',
 'ggg': 'G',
 'cat': 'H',
 'cac': 'H',
 'att': 'I',
 'atc': 'I',
 'ata': 'I',
 'aaa': 'K',
 'aag': 'K',
 'tta': 'L',
 'ttg': 'L',
 'ctt': 'L',
 'ctc': 'L',
 'cta': 'L',
 'ctg': 'L',
 'atg': 'M',
 'aat': 'N',
 'aac': 'N',
 'cct': 'P',
 'ccc': 'P',
 'cca': 'P',
 'ccg': 'P',
 'caa': 'Q',
 'cag': 'Q',
 'cgt': 'R',
 'cgc': 'R',
 'cga': 'R',
 'cgg': 'R',
 'aga': 'R',
 'agg': 'R',
 'tct': 'S',
 'tcc': 'S',
 'tca': 'S',
 'tcg': 'S',
 'agt': 'S',
 'agc': 'S',
 'act': 'T',
 'acc': 'T',
 'aca': 'T',
 'acg': 'T',
 'gtt': 'V',
 'gtc': 'V',
 'gta': 'V',
 'gtg': 'V',
 'tgg': 'W',
 'tat': 'Y',
 'tac': 'Y',
 'tga': 's',
 'taa': 's',
 'tag': 's'}

### 6) Lettura del file `EMBL` in un'unica stringa

*Suggerimento*: leggere il file in un'unica stringa usando il metodo `read()`.

In [14]:
with open(embl_file_name, 'r') as embl_file:
    embl_str = embl_file.read()

In [15]:
print(embl_str)

ID   M10051; SV 1; linear; mRNA; STD; HUM; 4723 BP.
XX
AC   M10051;
XX
DT   02-JUL-1986 (Rel. 09, Created)
DT   14-NOV-2006 (Rel. 89, Last updated, Version 7)
XX
DE   Human insulin receptor mRNA, complete cds.
XX
KW   insulin receptor; tyrosine kinase.
XX
OS   Homo sapiens (human)
OC   Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi; Mammalia;
OC   Eutheria; Euarchontoglires; Primates; Haplorrhini; Catarrhini; Hominidae;
OC   Homo.
XX
RN   [1]
RP   1-4723
RX   DOI; 10.1016/0092-8674(85)90334-4.
RX   PUBMED; 2859121.
RA   Ebina Y., Ellis L., Jarnagin K., Edery M., Graf L., Clauser E., Ou J.-H.,
RA   Masiarz F., Kan Y.W., Goldfine I.D., Roth R.A., Rutter W.J.;
RT   "The human insulin receptor cDNA: the structural basis for
RT   hormone-activated transmembrane signalling";
RL   Cell 40(4):747-758(1985).
XX
DR   MD5; e4e6ebf2e723a500c1dd62385c279351.
DR   Ensembl-Gn; ENSG00000171105; homo_sapiens.
DR   Ensembl-Tr; ENST00000302850; homo_sapiens.
DR   Ensembl-Tr; ENST0000034

### 7) Estrazione dell'identificatore univoco e dell'organismo relativo all'entry.

Estrarre dal *record* `ID`:

    ID   M10051; SV 1; linear; mRNA; STD; HUM; 4723 BP.
    
l'identificatore univoco `M10051` e l'organismo `HUM`.

In [17]:
identifier

'M10051'

In [18]:
organism

'HUM'

### 8) Estrazione della sequenza nucleotidica

a) Ottenere la lista dei *record* della sequenza nucleotidica, dopo avere escluso da ognuno gli spazi iniziali, gli spazi finali compreso l'intero.

    ggggggctgc gcggccgggt cggtgcgcac acgagaagga cgcgcggccc ccagcgctct        60

In [19]:
seq_row_list = re.findall('^\s+(.+?)\s+\d+', embl_str, re.M)

In [20]:
seq_row_list

['ggggggctgc gcggccgggt cggtgcgcac acgagaagga cgcgcggccc ccagcgctct',
 'tgggggccgc ctcggagcat gacccccgcg ggccagcgcc gcgcgcctga tccgaggaga',
 'ccccgcgctc ccgcagccat gggcaccggg ggccggcggg gggcggcggc cgcgccgctg',
 'ctggtggcgg tggccgcgct gctactgggc gccgcgggcc acctgtaccc cggagaggtg',
 'tgtcccggca tggatatccg gaacaacctc actaggttgc atgagctgga gaattgctct',
 'gtcatcgaag gacacttgca gatactcttg atgttcaaaa cgaggcccga agatttccga',
 'gacctcagtt tccccaaact catcatgatc actgattact tgctgctctt ccgggtctat',
 'gggctcgaga gcctgaagga cctgttcccc aacctcacgg tcatccgggg atcacgactg',
 'ttctttaact acgcgctggt catcttcgag atggttcacc tcaaggaact cggcctctac',
 'aacctgatga acatcacccg gggttctgtc cgcatcgaga agaacaatga gctctgttac',
 'ttggccacta tcgactggtc ccgtatcctg gattccgtgg aggataatca catcgtgttg',
 'aacaaagatg acaacgagga gtgtggagac atctgtccgg gtaccgcgaa gggcaagacc',
 'aactgccccg ccaccgtcat caacgggcag tttgtcgaac gatgttggac tcatagtcac',
 'tgccagaaag tttgcccgac catctgtaag tcacacggct gcaccgccga aggcctctgt',
 'tgccacagcg agtgcct

b) Concatenare le singole parti di sequenza (di 10 basi) per ottenere la sequenza nucleotidica in unica stringa senza spazi e in lettere maiuscole.

*Suggerimento*: il metodo `join()` degli oggetti `str` restituisce la concatenazione delle stringhe presenti nella lista passata come argomento usando la stringa invocante come separatore.

In [23]:
nucleotide_sequence = ''.join(seq_row_list).replace(' ', '')

In [24]:
nucleotide_sequence

'ggggggctgcgcggccgggtcggtgcgcacacgagaaggacgcgcggcccccagcgctcttgggggccgcctcggagcatgacccccgcgggccagcgccgcgcgcctgatccgaggagaccccgcgctcccgcagccatgggcaccgggggccggcggggggcggcggccgcgccgctgctggtggcggtggccgcgctgctactgggcgccgcgggccacctgtaccccggagaggtgtgtcccggcatggatatccggaacaacctcactaggttgcatgagctggagaattgctctgtcatcgaaggacacttgcagatactcttgatgttcaaaacgaggcccgaagatttccgagacctcagtttccccaaactcatcatgatcactgattacttgctgctcttccgggtctatgggctcgagagcctgaaggacctgttccccaacctcacggtcatccggggatcacgactgttctttaactacgcgctggtcatcttcgagatggttcacctcaaggaactcggcctctacaacctgatgaacatcacccggggttctgtccgcatcgagaagaacaatgagctctgttacttggccactatcgactggtcccgtatcctggattccgtggaggataatcacatcgtgttgaacaaagatgacaacgaggagtgtggagacatctgtccgggtaccgcgaagggcaagaccaactgccccgccaccgtcatcaacgggcagtttgtcgaacgatgttggactcatagtcactgccagaaagtttgcccgaccatctgtaagtcacacggctgcaccgccgaaggcctctgttgccacagcgagtgcctgggcaactgttctcagcccgacgaccccaccaagtgcgtggcctgccgcaacttctacctggacggcaggtgtgtggagacctgcccgcccccgtactaccacttccaggactggcgctgtgtgaacttcagcttctgccag

### 9) Estrazione della sequenza della proteina

a) Estrarre il prefisso della proteina che è contenuto nel *record*:

    FT                   /translation="MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLT

In [25]:
s = re.search('^FT\s+/translation=\"(\w+)', embl_str, re.M)
protein_prefix = s.group(1)

In [26]:
protein_prefix

'MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLT'

b) Costruire la lista di tutti gli altri record della proteina (compreso l'ultimo):

    FT                   RLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFP
    
**Attenzione all'ultimo**:

    FT                   DGGSSLGFKRSYEEHIPYTHMNGGKKNGRILTLPRSNPS"

**NB:** l'ultima termina con doppi apici `"`.

In [27]:
protein_list = re.findall('^FT\s+([A-Z]+)"?$', embl_str, re.M)

In [28]:
protein_list

['RLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFP',
 'NLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRI',
 'LDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTI',
 'CKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCV',
 'NFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCH',
 'LLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRR',
 'SYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTTTQGKLFFHYN',
 'PKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLRWE',
 'PYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHP',
 'GWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSS',
 'SQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDS',
 'QKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAE',
 'DPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGL',
 'RHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMW',
 'QEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSL',
 'AGNGSWTEPTYFYVTDYLDVPSNIAKIIIGPLIFVFLF

b) Aggiungere in testa alla lista il prefisso trovato prima e concatenare tutti gli elementi della lista per ottenere la sequenza della proteina in un'unica stringa.

In [29]:
protein_list[:0] = [protein_prefix]
protein_sequence = ''.join(protein_list)

In [30]:
protein_sequence

'MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFPNLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRILDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTICKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCVNFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCHLLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRRSYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTTTQGKLFFHYNPKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLRWEPYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHPGWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSSSQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDSQKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAEDPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGLRHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMWQEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSLAGNGSWTEPTYFYVTDYLDVPSNIAKIIIGPLIFVFLFSVVIGSIYLFLRKRQPDGPLGPLYASSNPEY

### 10) Determinazione della coding sequence (CDS)

a) Estrarre dal *record*

    FT   CDS             139..4287
    
lo start e l'end (1-based) della CDS.

In [32]:
cds_start

139

In [33]:
cds_end

4287

b) Estrarre la sequenza della CDS.

In [35]:
cds_sequence

'atgggcaccgggggccggcggggggcggcggccgcgccgctgctggtggcggtggccgcgctgctactgggcgccgcgggccacctgtaccccggagaggtgtgtcccggcatggatatccggaacaacctcactaggttgcatgagctggagaattgctctgtcatcgaaggacacttgcagatactcttgatgttcaaaacgaggcccgaagatttccgagacctcagtttccccaaactcatcatgatcactgattacttgctgctcttccgggtctatgggctcgagagcctgaaggacctgttccccaacctcacggtcatccggggatcacgactgttctttaactacgcgctggtcatcttcgagatggttcacctcaaggaactcggcctctacaacctgatgaacatcacccggggttctgtccgcatcgagaagaacaatgagctctgttacttggccactatcgactggtcccgtatcctggattccgtggaggataatcacatcgtgttgaacaaagatgacaacgaggagtgtggagacatctgtccgggtaccgcgaagggcaagaccaactgccccgccaccgtcatcaacgggcagtttgtcgaacgatgttggactcatagtcactgccagaaagtttgcccgaccatctgtaagtcacacggctgcaccgccgaaggcctctgttgccacagcgagtgcctgggcaactgttctcagcccgacgaccccaccaagtgcgtggcctgccgcaacttctacctggacggcaggtgtgtggagacctgcccgcccccgtactaccacttccaggactggcgctgtgtgaacttcagcttctgccaggacctgcaccacaaatgcaagaactcgcggaggcagggctgccaccaatacgtcattcacaacaacaagtgcatccctgagtgtccctccgggtacacgatgaattccagcaacttgctgtgcaccccatgcctgggt

### 11) Creazione della coding sequence (CDS) in formato `FASTA`

Produrre la sequenza della CDS in formato `FASTA` (separata in *record* di 60bp) con il seguente *header*:

    >M10051-HUM; len = [length]; start = [yes|no]; end = [yes|no]
    
e assegnarla alla variabile `cds_sequence_fasta`.

In [37]:
print(cds_sequence_fasta)

>M10051-HUM; len = 4149; start = yes; end = yes
atgggcaccgggggccggcggggggcggcggccgcgccgctgctggtggcggtggccgcg
ctgctactgggcgccgcgggccacctgtaccccggagaggtgtgtcccggcatggatatc
cggaacaacctcactaggttgcatgagctggagaattgctctgtcatcgaaggacacttg
cagatactcttgatgttcaaaacgaggcccgaagatttccgagacctcagtttccccaaa
ctcatcatgatcactgattacttgctgctcttccgggtctatgggctcgagagcctgaag
gacctgttccccaacctcacggtcatccggggatcacgactgttctttaactacgcgctg
gtcatcttcgagatggttcacctcaaggaactcggcctctacaacctgatgaacatcacc
cggggttctgtccgcatcgagaagaacaatgagctctgttacttggccactatcgactgg
tcccgtatcctggattccgtggaggataatcacatcgtgttgaacaaagatgacaacgag
gagtgtggagacatctgtccgggtaccgcgaagggcaagaccaactgccccgccaccgtc
atcaacgggcagtttgtcgaacgatgttggactcatagtcactgccagaaagtttgcccg
accatctgtaagtcacacggctgcaccgccgaaggcctctgttgccacagcgagtgcctg
ggcaactgttctcagcccgacgaccccaccaagtgcgtggcctgccgcaacttctacctg
gacggcaggtgtgtggagacctgcccgcccccgtactaccacttccaggactggcgctgt
gtgaacttcagcttctgccaggacctgcaccacaaatgcaagaactcgcggaggcagggc
tgccaccaatacgtcattcacaacaacaagtgcatcc

Stampare la coding sequence in un file.

### 12) Determinazione delle frequenze dei codoni

a) Estrarre la lista dei codoni della CDS.

In [40]:
codon_list

['atg',
 'ggc',
 'acc',
 'ggg',
 'ggc',
 'cgg',
 'cgg',
 'ggg',
 'gcg',
 'gcg',
 'gcc',
 'gcg',
 'ccg',
 'ctg',
 'ctg',
 'gtg',
 'gcg',
 'gtg',
 'gcc',
 'gcg',
 'ctg',
 'cta',
 'ctg',
 'ggc',
 'gcc',
 'gcg',
 'ggc',
 'cac',
 'ctg',
 'tac',
 'ccc',
 'gga',
 'gag',
 'gtg',
 'tgt',
 'ccc',
 'ggc',
 'atg',
 'gat',
 'atc',
 'cgg',
 'aac',
 'aac',
 'ctc',
 'act',
 'agg',
 'ttg',
 'cat',
 'gag',
 'ctg',
 'gag',
 'aat',
 'tgc',
 'tct',
 'gtc',
 'atc',
 'gaa',
 'gga',
 'cac',
 'ttg',
 'cag',
 'ata',
 'ctc',
 'ttg',
 'atg',
 'ttc',
 'aaa',
 'acg',
 'agg',
 'ccc',
 'gaa',
 'gat',
 'ttc',
 'cga',
 'gac',
 'ctc',
 'agt',
 'ttc',
 'ccc',
 'aaa',
 'ctc',
 'atc',
 'atg',
 'atc',
 'act',
 'gat',
 'tac',
 'ttg',
 'ctg',
 'ctc',
 'ttc',
 'cgg',
 'gtc',
 'tat',
 'ggg',
 'ctc',
 'gag',
 'agc',
 'ctg',
 'aag',
 'gac',
 'ctg',
 'ttc',
 'ccc',
 'aac',
 'ctc',
 'acg',
 'gtc',
 'atc',
 'cgg',
 'gga',
 'tca',
 'cga',
 'ctg',
 'ttc',
 'ttt',
 'aac',
 'tac',
 'gcg',
 'ctg',
 'gtc',
 'atc',
 'ttc',
 'gag',
 'atg',


b) Costruire la lista di tuple *(codone, frequenza)* elencate per frequenze decrescenti.

In [42]:
codon_frequency

[('gag', 77),
 ('ctg', 67),
 ('aac', 53),
 ('gtg', 49),
 ('aag', 47),
 ('gac', 44),
 ('atc', 41),
 ('acc', 39),
 ('ttc', 39),
 ('ggc', 37),
 ('ccc', 37),
 ('ctc', 35),
 ('cag', 32),
 ('cac', 31),
 ('atg', 30),
 ('tgc', 30),
 ('tac', 29),
 ('gaa', 29),
 ('ggg', 28),
 ('gcc', 28),
 ('gtc', 28),
 ('tcc', 27),
 ('gat', 26),
 ('cgg', 25),
 ('att', 23),
 ('tct', 22),
 ('aaa', 21),
 ('tat', 21),
 ('ttt', 20),
 ('tgg', 20),
 ('aat', 19),
 ('agg', 18),
 ('gga', 17),
 ('tgt', 17),
 ('ttg', 17),
 ('agt', 17),
 ('gcg', 16),
 ('acg', 16),
 ('cct', 16),
 ('tca', 15),
 ('cca', 15),
 ('ccg', 14),
 ('cga', 14),
 ('cgc', 13),
 ('agc', 12),
 ('tcg', 12),
 ('ggt', 11),
 ('gct', 11),
 ('act', 10),
 ('gca', 10),
 ('gtt', 8),
 ('ctt', 8),
 ('cat', 7),
 ('aga', 7),
 ('cta', 6),
 ('cgt', 5),
 ('caa', 5),
 ('aca', 5),
 ('tta', 3),
 ('gta', 2),
 ('ata', 1),
 ('taa', 1)]

### 13) Determinazione delle frequenze degli amminoacidi della proteina letta dal file `EMBL`

a) Estrarre la lista degli amminoacidi della proteina.

In [44]:
ammino_list

['M',
 'G',
 'T',
 'G',
 'G',
 'R',
 'R',
 'G',
 'A',
 'A',
 'A',
 'A',
 'P',
 'L',
 'L',
 'V',
 'A',
 'V',
 'A',
 'A',
 'L',
 'L',
 'L',
 'G',
 'A',
 'A',
 'G',
 'H',
 'L',
 'Y',
 'P',
 'G',
 'E',
 'V',
 'C',
 'P',
 'G',
 'M',
 'D',
 'I',
 'R',
 'N',
 'N',
 'L',
 'T',
 'R',
 'L',
 'H',
 'E',
 'L',
 'E',
 'N',
 'C',
 'S',
 'V',
 'I',
 'E',
 'G',
 'H',
 'L',
 'Q',
 'I',
 'L',
 'L',
 'M',
 'F',
 'K',
 'T',
 'R',
 'P',
 'E',
 'D',
 'F',
 'R',
 'D',
 'L',
 'S',
 'F',
 'P',
 'K',
 'L',
 'I',
 'M',
 'I',
 'T',
 'D',
 'Y',
 'L',
 'L',
 'L',
 'F',
 'R',
 'V',
 'Y',
 'G',
 'L',
 'E',
 'S',
 'L',
 'K',
 'D',
 'L',
 'F',
 'P',
 'N',
 'L',
 'T',
 'V',
 'I',
 'R',
 'G',
 'S',
 'R',
 'L',
 'F',
 'F',
 'N',
 'Y',
 'A',
 'L',
 'V',
 'I',
 'F',
 'E',
 'M',
 'V',
 'H',
 'L',
 'K',
 'E',
 'L',
 'G',
 'L',
 'Y',
 'N',
 'L',
 'M',
 'N',
 'I',
 'T',
 'R',
 'G',
 'S',
 'V',
 'R',
 'I',
 'E',
 'K',
 'N',
 'N',
 'E',
 'L',
 'C',
 'Y',
 'L',
 'A',
 'T',
 'I',
 'D',
 'W',
 'S',
 'R',
 'I',
 'L',
 'D',
 'S',
 'V'

a) Costruire la lista di tuple *(amminoacido, frequenza)* elencate per frequenza decrescente.

In [46]:
ammino_frequency

[('L', 136),
 ('E', 106),
 ('S', 105),
 ('G', 93),
 ('V', 87),
 ('R', 82),
 ('P', 82),
 ('N', 72),
 ('T', 70),
 ('D', 70),
 ('K', 68),
 ('A', 65),
 ('I', 65),
 ('F', 59),
 ('Y', 50),
 ('C', 47),
 ('H', 38),
 ('Q', 37),
 ('M', 30),
 ('W', 20)]

b) Produrre il diagramma a barre delle frequenze degli amminoacidi.

Importare il package `matplotlib`

Importare il modulo `pyplot`.

Creare la lista dei simboli di amminoacido (asse x del diagramma)

In [61]:
ammino_acid_list

['L',
 'E',
 'S',
 'G',
 'V',
 'R',
 'P',
 'N',
 'T',
 'D',
 'K',
 'A',
 'I',
 'F',
 'Y',
 'C',
 'H',
 'Q',
 'M',
 'W']

Creare la corrispondente lista delle frequenze (asse y del diagramma)

In [63]:
freq_list

[136,
 106,
 105,
 93,
 87,
 82,
 82,
 72,
 70,
 70,
 68,
 65,
 65,
 59,
 50,
 47,
 38,
 37,
 30,
 20]

### 14) Validazione della sequenza della proteina letta dal file `EMBL`

a) Tradurre in proteina la sequenza della CDS.

**Alternativa1:** traduzione della lista `codon_list` da lista di codoni a lista di amminoacidi e unione in unica stringa con il metodo `join()`.

In [69]:
cds_translation

'MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFPNLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRILDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTICKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCVNFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCHLLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRRSYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTTTQGKLFFHYNPKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLRWEPYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHPGWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSSSQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDSQKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAEDPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGLRHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMWQEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSLAGNGSWTEPTYFYVTDYLDVPSNIAKIIIGPLIFVFLFSVVIGSIYLFLRKRQPDGPLGPLYASSNPEY

**Alternativa2:** traduzione dalla stringa della coding sequence con la funzione `sub()` del modulo `re`.

La funzione:

    re.sub(my_expr, r_arg, my_string)
    
sostituisce tutte le occorrenze non sovrapposte della *RE* `my_expr` in `my_string` con l'argomento `r_arg`.

In [71]:
cds_translation2

'MGTGGRRGAAAAPLLVAVAALLLGAAGHLYPGEVCPGMDIRNNLTRLHELENCSVIEGHLQILLMFKTRPEDFRDLSFPKLIMITDYLLLFRVYGLESLKDLFPNLTVIRGSRLFFNYALVIFEMVHLKELGLYNLMNITRGSVRIEKNNELCYLATIDWSRILDSVEDNHIVLNKDDNEECGDICPGTAKGKTNCPATVINGQFVERCWTHSHCQKVCPTICKSHGCTAEGLCCHSECLGNCSQPDDPTKCVACRNFYLDGRCVETCPPPYYHFQDWRCVNFSFCQDLHHKCKNSRRQGCHQYVIHNNKCIPECPSGYTMNSSNLLCTPCLGPCPKVCHLLEGEKTIDSVTSAQELRGCTVINGSLIINIRGGNNLAAELEANLGLIEEISGYLKIRRSYALVSLSFFRKLRLIRGETLEIGNYSFYALDNQNLRQLWDWSKHNLTTTQGKLFFHYNPKLCLSEIHKMEEVSGTKGRQERNDIALKTNGDKASCENELLKFSYIRTSFDKILLRWEPYWPPDFRDLLGFMLFYKEAPYQNVTEFDGQDACGSNSWTVVDIDPPLRSNDPKSQNHPGWLMRGLKPWTQYAIFVKTLVTFSDERRTYGAKSDIIYVQTDATNPSVPLDPISVSNSSSQIILKWKPPSDPNGNITHYLVFWERQAEDSELFELDYCLKGLKLPSRTWSPPFESEDSQKHNQSEYEDSAGECCSCPKTDSQILKELEESSFRKTFEDYLHNVVFVPRKTSSGTGAEDPRPSRKRRSLGDVGNVTVAVPTVAAFPNTSSTSVPTSPEEHRPFEKVVNKESLVISGLRHFTGYRIELQACNQDTPEERCSVAAYVSARTMPEAKADDIVGPVTHEIFENNVVHLMWQEPKEPNGLIVLYEVSYRRYGDEELHLCVSRKHFALERGCRLRGLSPGNYSVRIRATSLAGNGSWTEPTYFYVTDYLDVPSNIAKIIIGPLIFVFLFSVVIGSIYLFLRKRQPDGPLGPLYASSNPEY

Verificare che le due alternative portano alla stessa traduzione.

True

Verificare infine che la proteina letta dal file `EMBL` è uguale a quella ottenuta per traduzione della CDS.

True

### 15) Trovare una CDS "sinonima" della precedente ottenuta sostituendo il maggior numero di codoni

a) Costruire il dizionario inverso del codice genetico:

- *chiave*: simbolo di amminoacido
- *valore*: lista dei codoni che corrispondono all'amminoacido

In [77]:
inverse_genetic_code_dict

{'A': ['gct', 'gcc', 'gca', 'gcg'],
 'C': ['tgt', 'tgc'],
 'D': ['gat', 'gac'],
 'E': ['gaa', 'gag'],
 'F': ['ttt', 'ttc'],
 'G': ['ggt', 'ggc', 'gga', 'ggg'],
 'H': ['cat', 'cac'],
 'I': ['att', 'atc', 'ata'],
 'K': ['aaa', 'aag'],
 'L': ['tta', 'ttg', 'ctt', 'ctc', 'cta', 'ctg'],
 'M': ['atg'],
 'N': ['aat', 'aac'],
 'P': ['cct', 'ccc', 'cca', 'ccg'],
 'Q': ['caa', 'cag'],
 'R': ['cgt', 'cgc', 'cga', 'cgg', 'aga', 'agg'],
 'S': ['tct', 'tcc', 'tca', 'tcg', 'agt', 'agc'],
 'T': ['act', 'acc', 'aca', 'acg'],
 'V': ['gtt', 'gtc', 'gta', 'gtg'],
 'W': ['tgg'],
 'Y': ['tat', 'tac'],
 's': ['tga', 'taa', 'tag']}

b) Determinare una CDS sinonima sostituendo il maggior numero possibile di codoni.

In [79]:
syn_cds_sequence

'atgggtactggtggtcgtcgtggtgctgctgctgctcctttattagttgctgttgctgctttattattaggtgctgctggtcatttatatcctggtgaagtttgccctggtatggacattcgtaataatttaacccgtttacacgaattagaaaactgttccgttattgagggtcatttacaaattttattaatgtttaagactcgtcctgaggactttcgtgatttatcttttcctaagttaattatgattaccgactatttattattatttcgtgtttacggtttagaatctttaaaagatttatttcctaatttaactgttattcgtggttctcgtttatttttcaattatgctttagttatttttgaaatggtccatttaaaagagttaggtttatataatttaatgaatattactcgtggctccgttcgtattgaaaaaaataacgaattatgctatttagctaccattgattggtctcgcattttagactctgttgaagacaaccatattgttttaaataaggacgataatgaagaatgcggtgatatttgccctggcactgctaaaggtaaaactaattgtcctgctactgttattaatggtcaattcgttgagcgttgctggacccactctcattgtcaaaaggtctgtcctactatttgcaaatctcatggttgtactgctgagggtttatgctgtcattctgaatgtttaggtaattgctcccaacctgatgatcctactaaatgtgttgcttgtcgtaatttttatttagatggtcgttgcgttgaaacttgtcctcctccttattatcattttcaagattggcgttgcgttaatttttctttttgtcaagatttacatcataagtgtaaaaattctcgtcgtcaaggttgtcatcagtatgttatccataataataaatgtattcccgaatgcccttctggttatactatgaactcttctaatttattatgtactccttgtttaggc

c) Verificare che fornisca la stessa proteina.

True

d) Misurare la differenza con la precedente CDS tramite distanza di Hamming, che è il numero di posizioni in cui le due sequenze hanno diverso carattere.

**Esempio**: la distanza di Hamming di `ACGTG` e `GCTTG` è pari a 2, in quanto le basi diverse sono quelle in posizione 1 (`A` e `G`) e quelle in posizione 3 (`G` e `T`).

*Suggerimento*: usare la funzione `range()` passando come unico argomento la lunghezza della CDS, per produrre il range dei suoi indici di posizione. 

In [82]:
hamming_dist

1509

Percentuale di basi differenti rispetto alla lunghezza delle due CDS:

0.36370209689081706

Numero di possibili CDS che esprimono la stessa proteina:

In [85]:
count

108882884128409166024746492002735855651428001792320127244126386647029980632289513784045044140581393841618827221748361594073355650045240363812106073841182979165897587490232446777783983313880563133730376849029385911803929480037903720952494698441685495017262385109095191569323097956789003231766319113433010022112570580209761930707000543314601534394699157328283463123684349487972283113201441194551015153866798709856868536638771549717606971376688439270202789826226688266419863942529413908709032123007393702560744391321515523540532123485323124014154147380386166265468444631304419682081513203797302729799078430988163220003146226946076604133339594066640842997415150550171343877731740457868722176